In [1]:
import os
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

from analysis_utils import *

%load_ext autoreload
%autoreload 2

In [2]:
csv_file_path = "ray_osdi_12_8_night_sysml_02_homo_fix_gamma_w_dynamic_run_2/alibaba_scheduler_TetriSched_release_policy_fixed_gamma_max_deadline_var_200_dag_aware_1_poisson_arrival_rate_0.06_gamma_coefficient_1_base_arrival_rate_0.03_scheduler_discretization_1_dynamic_max_occupancy_threshold_0.7_noCPOptPass/alibaba_scheduler_TetriSched_release_policy_fixed_gamma_max_deadline_var_200_dag_aware_1_poisson_arrival_rate_0.06_gamma_coefficient_1_base_arrival_rate_0.03_scheduler_discretization_1_dynamic_max_occupancy_threshold_0.7_noCPOptPass.csv"
csv_reader = CSVReader([csv_file_path])

worker_pools = csv_reader.get_worker_pools(csv_file_path)
task_graphs = csv_reader.get_task_graph(csv_file_path).values()
tasks = csv_reader.get_tasks(csv_file_path)

df_task_graphs = pd.DataFrame(task_graphs)
df_tasks =  pd.DataFrame(tasks)

In [3]:
for worker_pool in worker_pools:
    print(worker_pool.resources)

[Resource(name='Slot_1', id='bdd640fb-0667-4ad1-9c80-317fa3b1799d', quantity='10')]


In [4]:
completed_task_graph_run_time, canceled_task_graph_run_time, miss_deadline_task_graph_run_time = csv_reader.get_time_spent_on_completed_canceled_miss_deadline_task_graph(csv_file_path)
print(f"Total time spent on completed task graphs: {sum(completed_task_graph_run_time.values())}")
print(f"Total time spent on canceled task graphs: {sum(canceled_task_graph_run_time.values())}")
print(f"Total time spent on miss_deadline task graphs: {sum(miss_deadline_task_graph_run_time.values())}")

Total time spent on completed task graphs: 7205
Total time spent on canceled task graphs: 39
Total time spent on miss_deadline task graphs: 0


In [5]:
df_task_graphs.describe()

,release_time,deadline,num_tasks,window_to_execute,cancelled_at,completion_time,slack
count,50.000000,50.000000,50.000000,50.000000,20.000000,30.000000,30.000000
mean,370.820000,658.840000,5.920000,288.020000,415.900000,697.566667,3.100000
std,148.377441,549.495552,4.826965,514.505211,141.436205,613.997286,11.317822
min,81.000000,88.000000,2.000000,2.000000,81.000000,108.000000,0.000000
25%,254.000000,330.000000,3.000000,9.000000,325.500000,317.250000,0.000000
50%,396.500000,471.000000,4.000000,40.000000,435.500000,496.000000,0.000000
75%,503.750000,720.250000,7.750000,292.750000,531.250000,882.250000,0.750000
max,601.000000,2695.000000,26.000000,2406.000000,601.000000,2695.000000,61.000000


In [6]:
df_tasks.describe()

,timestamp,intended_release_time,release_time,deadline,window_to_execute,runtime,start_time,placement_time,completion_time,slack,cancelled_at
count,257.0,257.000000,257.000000,257.000000,257.000000,230.000000,230.000000,230.000000,230.000000,230.000000,27.000000
mean,1.0,165.256809,559.463035,902.673152,343.210117,31.495652,647.869565,647.869565,679.365217,263.278261,421.740741
std,0.0,209.747868,486.743146,772.579452,525.070169,57.664119,550.274001,550.274001,587.481433,421.908081,133.749074
min,1.0,-1.000000,81.000000,88.000000,1.000000,1.000000,81.000000,81.000000,82.000000,0.000000,81.000000
25%,1.0,-1.000000,256.000000,339.000000,26.000000,1.000000,273.750000,273.750000,288.500000,12.000000,325.000000
50%,1.0,-1.000000,455.000000,561.000000,120.000000,7.000000,523.000000,523.000000,525.500000,82.000000,462.000000
75%,1.0,379.000000,571.000000,1141.000000,486.000000,27.750000,696.750000,696.750000,720.500000,331.500000,532.000000
max,1.0,601.000000,2580.000000,2695.000000,2406.000000,404.000000,2580.000000,2580.000000,2695.000000,2405.000000,601.000000


In [7]:
df_task_graphs_cancelled = df_task_graphs[df_task_graphs["cancelled"] == True].drop(["completion_time", "slack"], axis=1)
print("For the task graphs that get canceled, what are they?")   
print(df_task_graphs_cancelled.describe())
df_task_graphs_cancelled

For the task graphs that get canceled, what are they?
       release_time     deadline  num_tasks  window_to_execute  cancelled_at
count     20.000000    20.000000  20.000000          20.000000     20.000000
mean     380.400000   596.100000   3.800000         215.700000    415.900000
std      132.496832   440.232941   2.706717         440.855042    141.436205
min       81.000000    88.000000   2.000000           2.000000     81.000000
25%      319.750000   355.500000   2.000000           4.750000    325.500000
50%      401.000000   442.500000   3.000000          10.000000    435.500000
75%      468.000000   629.500000   4.000000          77.750000    531.250000
max      587.000000  1713.000000  11.000000        1427.000000    601.000000


,name,release_time,deadline,num_tasks,window_to_execute,cancelled,cancelled_at,deadline_miss_detected_at
0,j_977163@1,81,88,2,7,True,81.0,None
8,j_1244951@1,204,211,2,7,True,204.0,None
9,j_1452016@1,206,208,2,2,True,206.0,None
11,j_3184340@1,230,1657,4,1427,True,531.0,None
16,j_785518@1,307,327,4,20,True,307.0,None
18,j_2645786@1,324,327,2,3,True,324.0,None
19,j_3442797@1,326,373,5,47,True,326.0,None
20,j_3991859@1,344,1713,3,1369,True,548.0,None
21,j_3645112@1,358,365,2,7,True,358.0,None
25,j_3638170@1,400,411,2,11,True,400.0,None


In [20]:
df_task_graphs_completed = df_task_graphs[df_task_graphs["cancelled"] == False].drop(["cancelled_at", "deadline_miss_detected_at"], axis=1)
print("For the task graphs that get completed, what are they?")   
print(df_task_graphs_completed.describe())
df_task_graphs_completed

For the task graphs that get completed, what are they?
       release_time     deadline  num_tasks  completion_time      slack
count     30.000000    30.000000  30.000000        30.000000  30.000000
mean     364.433333   700.666667   7.333333       697.566667   3.100000
std      159.974714   615.372516   5.422389       613.997286  11.317822
min       81.000000   108.000000   2.000000       108.000000   0.000000
25%      227.750000   317.250000   3.000000       317.250000   0.000000
50%      386.000000   496.500000   5.500000       496.000000   0.000000
75%      519.500000   928.000000   9.750000       882.250000   0.750000
max      601.000000  2695.000000  26.000000      2695.000000  61.000000


,name,release_time,deadline,num_tasks,cancelled,completion_time,slack
1,j_2345487@1,81,108,12,False,108.0,0.0
2,j_676668@1,115,410,3,False,410.0,0.0
3,j_2212258@1,121,215,6,False,215.0,0.0
4,j_378922@1,150,162,5,False,162.0,0.0
5,j_4061192@1,160,168,2,False,167.0,1.0
6,j_650891@1,185,471,8,False,471.0,0.0
7,j_2962188@1,200,306,14,False,306.0,0.0
10,j_2413863@1,219,339,14,False,339.0,0.0
12,j_518111@1,254,289,2,False,284.0,5.0
13,j_2740033@1,254,261,3,False,261.0,0.0


In [11]:
df_tasks_cancelled = df_tasks[df_tasks["cancelled"] == True].drop(["completion_time", "runtime", "intended_release_time", "skipped_times", "missed_deadline", "deadline_miss_detected_at", "timestamp", "placements"], axis=1)
print("For the tasks that get canceled, what are they?")
print(df_tasks_cancelled.describe())
df_tasks_cancelled.sort_values(by=["release_time"]).drop(["task_id"], axis=1)

For the tasks that get canceled, what are they?
       release_time     deadline  window_to_execute  start_time  \
count     27.000000    27.000000          27.000000         0.0   
mean     396.666667   562.185185         165.518519         NaN   
std      130.683116   384.905087         386.443087         NaN   
min       81.000000    88.000000           2.000000         NaN   
25%      315.500000   346.000000           7.000000         NaN   
50%      402.000000   471.000000          20.000000         NaN   
75%      532.000000   561.000000          38.000000         NaN   
max      593.000000  1713.000000        1427.000000         NaN   

       placement_time  slack  cancelled_at  
count             0.0    0.0     27.000000  
mean              NaN    NaN    421.740741  
std               NaN    NaN    133.749074  
min               NaN    NaN     81.000000  
25%               NaN    NaN    325.000000  
50%               NaN    NaN    462.000000  
75%               NaN    NaN    5

,name,task_graph,release_time,deadline,window_to_execute,start_time,placement_time,slack,cancelled,cancelled_at
2,M1,j_977163@1,81,88,7,NaN,NaN,NaN,True,81.0
38,M1,j_1244951@1,204,211,7,NaN,NaN,NaN,True,204.0
39,M1,j_1452016@1,206,208,2,NaN,NaN,NaN,True,206.0
54,M1,j_3184340@1,230,1657,1427,NaN,NaN,NaN,True,531.0
78,M1,j_785518@1,307,327,20,NaN,NaN,NaN,True,307.0
79,M2,j_785518@1,307,327,20,NaN,NaN,NaN,True,307.0
80,M3,j_785518@1,307,327,20,NaN,NaN,NaN,True,307.0
84,M1,j_2645786@1,324,327,3,NaN,NaN,NaN,True,324.0
85,M1,j_3442797@1,326,373,47,NaN,NaN,NaN,True,326.0
86,M3,j_3442797@1,326,373,47,NaN,NaN,NaN,True,326.0


In [27]:
df_tasks_cancelled = df_tasks[df_tasks["cancelled"] == False].drop(["timestamp", "placements", "skipped_times", "deadline_miss_detected_at", "intended_release_time", "cancelled", "cancelled_at"], axis=1)
print("For the tasks that get completed, what are they?")
print(df_tasks_cancelled.describe())
df_tasks_cancelled.sort_values(by=["window_to_execute"]).drop(["task_id"], axis=1)

For the tasks that get completed, what are they?
       release_time     deadline  window_to_execute     runtime   start_time  \
count    230.000000   230.000000         230.000000  230.000000   230.000000   
mean     578.573913   942.643478         364.069565   31.495652   647.869565   
std      509.334836   796.968757         535.807292   57.664119   550.274001   
min       81.000000   108.000000           1.000000    1.000000    81.000000   
25%      248.000000   339.000000          30.000000    1.000000   273.750000   
50%      478.500000   579.000000         127.500000    7.000000   523.000000   
75%      580.750000  1141.000000         486.000000   27.750000   696.750000   
max     2580.000000  2695.000000        2406.000000  404.000000  2580.000000   

       placement_time  completion_time        slack  
count      230.000000       230.000000   230.000000  
mean       647.869565       679.365217   263.278261  
std        550.274001       587.481433   421.908081  
min         81

,name,task_graph,release_time,deadline,window_to_execute,runtime,start_time,placement_time,completion_time,slack,missed_deadline
233,R7,j_3779754@1,1207,1208,1,1.0,1207.0,1207.0,1208.0,0.0,False
66,R3,j_2740033@1,260,261,1,1.0,260.0,260.0,261.0,0.0,False
26,R5,j_378922@1,161,162,1,1.0,161.0,161.0,162.0,0.0,False
22,J4,j_378922@1,160,162,2,1.0,160.0,160.0,161.0,1.0,False
76,J14,j_2962188@1,304,306,2,2.0,304.0,304.0,306.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...
68,M16,j_3572397@1,289,2695,2406,39.0,1322.0,1322.0,1361.0,1334.0,False
71,M19,j_3572397@1,289,2695,2406,1.0,289.0,289.0,290.0,2405.0,False
70,M18,j_3572397@1,289,2695,2406,12.0,1294.0,1294.0,1306.0,1389.0,False
72,M21,j_3572397@1,289,2695,2406,33.0,349.0,349.0,382.0,2313.0,False


In [12]:
print("Why the hell is task get cancelled at time = 81?")
df_tasks[df_tasks["release_time"] == 81].sort_values(by=["window_to_execute"]).drop(["timestamp", "intended_release_time", "task_id", "placement_time", "skipped_times"], axis=1)

Why the hell is task get cancelled at time = 81?


,name,task_graph,release_time,deadline,window_to_execute,runtime,placements,start_time,completion_time,slack,missed_deadline,deadline_miss_detected_at,cancelled,cancelled_at
2,M1,j_977163@1,81,88,7,NaN,[],NaN,NaN,NaN,False,None,True,81.0
0,M10,j_2345487@1,81,108,27,1.0,"[{'task_name': 'M10', 'timestamp': 1, 'task_id...",84.0,85.0,23.0,False,None,False,NaN
1,M1,j_2345487@1,81,108,27,1.0,"[{'task_name': 'M1', 'timestamp': 1, 'task_id'...",83.0,84.0,24.0,False,None,False,NaN
3,M4,j_2345487@1,81,108,27,1.0,"[{'task_name': 'M4', 'timestamp': 1, 'task_id'...",81.0,82.0,26.0,False,None,False,NaN
4,M7,j_2345487@1,81,108,27,12.0,"[{'task_name': 'M7', 'timestamp': 1, 'task_id'...",84.0,96.0,12.0,False,None,False,NaN
5,M8,j_2345487@1,81,108,27,1.0,"[{'task_name': 'M8', 'timestamp': 1, 'task_id'...",92.0,93.0,15.0,False,None,False,NaN
